In [0]:
import cv2
from keras.utils import Sequence
import pandas as pd
import sys
sys.path.append('../')
from model.model import vgg16Model, vqaModelBiLSTMFeatures
from utils.utils import *
from keras.models import model_from_json

In [0]:
def load_vqa_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weight_path)
#     model = None
#     if model_name == 'vqaModelBiLSTMFeatures':
#         model = vqaModelBiLSTMFeatures(embedding_matrix, image_feature_dims=4096, trainable=False, num_classes=1000,embed_size=300, vocab_size=vocab_size, time_steps=20, unit_length=256, dropout=0.5)
    return loaded_model

In [0]:
def getImageFeatures(image, model):
    im = cv2.resize(image, (224,224))
    im = im.reshape((1,224,224,3))
    im = im/255.0
    features = model.predict(im)
    return features

In [0]:
def predictClass(question, image, time_steps):
    global tokenizer, le, vgg16, vgaModel
    image_features = getImageFeatures(image, vgg16)
    question_sequence, question_padded = createEncodedPaddedText(tokenizer, question,time_steps=time_steps)
    prediction = vqaModel.predict([np.asarray(question_padded), np.asarray(image_features)])
    answer_class = prediction.argmax(axis=1)
    answer = le.inverse_transform(answer_class)
    return answer

In [0]:
vgg16 = vgg16Model(1000)

In [9]:
vqaModel = load_vqa_model('../extras/vqaModelBiLSTMFeatures.json', '../weights/best_weights.h5')


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#Read tokenizer
tokenizer = readFromPickle('../extras/', 'tokenizer')
word_index = tokenizer.word_index
#Read label encoder
le = readFromPickle('../extras/', 'labelEncoder')

In [110]:
image = cv2.imread('pizza.jpeg')
predictClass(['what is the color of the pizza?'], image, 20)

array(['white and blue'], dtype=object)